# Integration with the scientific Python ecosystem 🐍

In this tutorial, we'll try out the integration between PyGMT and other common packages in the scientific Python ecosystem.


Besides [pygmt](https://www.pygmt.org), we'll also be using:

- [GeoPandas](https://geopandas.org/en/stable/) for managing geospatial tabular data
- [Panel](https://panel.holoviz.org/index.html) for interactive visualizations
- [Xarray](https://xarray.dev/) for managing n-dimensional labelled arrays


## Plotting geospatial vector data with GeoPandas and PyGMT

We'll extend the GeoPandas [Mapping and Plotting Tools Examples](https://geopandas.org/en/stable/docs/user_guide/mapping.html) to show how to create choropleth maps using PyGMT.

**References**:

 - GeoPandas User Guide - https://geopandas.org/en/stable/docs/user_guide/

In [ ]:
import pygmt
import geopandas as gpd

We'll load sample data provided through the GeoPandas package and inspect the GeoDataFrame.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

Following the [GeoPandas example](https://geopandas.org/en/stable/docs/user_guide/mapping.html#choropleth-maps), we'll create a Choropleth map showing world population estimates, but will use PyGMT to plot the data using the [Hammer projection](https://www.pygmt.org/latest/projections/misc/misc_hammer.html#hammer).

In [ ]:
# Calculate the populations in millions per capita
world = world[(world.pop_est>0) & (world.name!="Antarctica")]
world['pop_est'] = world.pop_est * 1e-6

# Find the range of data values for creating a colormap
cmap_bounds = pygmt.info(data=world['pop_est'], per_column=True)
cmap_bounds

Now, we'll plot the data on a PyGMT figure, by creating a figure instance, laying down a basemap, plotting the GeoDataFrame, and adding a colorbar!

In [ ]:
# Create an instance of the pygmt.Figure class
fig = pygmt.Figure()
# Create a colormap for the figure
pygmt.makecpt(cmap="bilbao", series=cmap_bounds)
# Create a basemap
fig.basemap(region="d", projection="H15c", frame=True)
# Plot the GeoDataFrame
# - Use `close=True` to specify that the polygons should be forced closed
# - Plot the polygon outlines with a 1 point, black pen
# - Set that the color should be based on the `pop_est` using the `color, `cmap`, and `aspatial` parameters
fig.plot(data=world, pen="1p,black", close=True, color="+z", cmap=True, aspatial="Z=pop_est")
# Add a colorbar
fig.colorbar(position="JMR", frame='a200+lPopulation (millions)')
# Display the output
fig.show()


## Interactive data visualization with Xarray, Panel, and PyGMT

In this section, we'll create some interactive visualizations of oceanographic data!

We'll use [Panel](https://panel.holoviz.org/index.html), which is a Python library
for connecting interactive widgets with plots! We'll use Panel with
[PyGMT](https://www.pygmt.org) and [xarray](https://www.xarray.dev) to visualize
the objectively interpolated mean field for in-situ temperature from the World Ocean Atlas.

**References**:

- Temperature visualization based on https://rabernat.github.io/intro_to_physical_oceanography/02-c_ocean_temperature_salinity_stratification.html
- Interactive setup based on https://github.com/weiji14/30DayMapChallenge2021/blob/main/day25_interactive.py
- Data from the NOAA World Ocean Atlas, stored on the IRI Data Library at http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NODC/.WOA09/.

In [ ]:
import panel as pn
import xarray as xr
import pygmt
pn.extension()


In [ ]:
# Download the dataset from the IRI Data Library
url = 'https://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NODC/.WOA09/.Grid-1x1/.Annual/.temperature/.t_an/data.nc'
netcdf_file = pygmt.which(fname=url, download=True)
woa_temp = xr.open_dataset(netcdf_file).isel(time=0)
woa_temp

In [ ]:
# Make a static plot of sea surface temperature
fig = pygmt.Figure()
fig.grdimage(grid=woa_temp.t_an.sel(depth=0), cmap="vik", projection="R15c", frame=True)
fig.show()

In [ ]:
# Make a panel widget for controlling the depth plotted
depth_slider = pn.widgets.DiscreteSlider(name='Depth (m)', options=woa_temp.depth.values.astype(int).tolist(), value=0)

In [ ]:
# Make a function for plotting the depth slice with PyGMT

@pn.depends(depth=depth_slider)
def view(depth: int):
    fig = pygmt.Figure()
    pygmt.makecpt(cmap="vik", series=[-2,30])
    fig.grdimage(grid=woa_temp.t_an.sel(depth=depth), cmap=True, projection="R15c", frame=True)
    fig.colorbar(frame="a5")
    return fig

### Make the interactive dashboard!

Now to put everything together! The 'dashboard' will be very simple.
The 'depth' slider is placed next to the map using `panel.Column`.
Selecting different depths will update the data plotted! Find out more at
https://panel.holoviz.org/getting_started/index.html#using-panel.

Note: This is meant to run in a Jupyter lab/notebook environment.
The grdinfo warning can be ignored.

In [ ]:
pn.Column(depth_slider, view)